In [1]:
import keras as k
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

import random

C:\Users\ortci\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import numpy as np

n_samples = 1000000
n_partitions = 10

# Generate tasks and partitions
partition_data = np.random.random((n_samples, n_partitions)) # partition data generation

data = np.zeros((n_samples, 2*n_partitions))                # initialize input layer
labels = np.zeros((n_samples, n_partitions))                 # initialize outputs layer for training 

task_data = np.zeros((n_samples, 1))                         # initialize task list

# fails to account a 'perfect' fit where fit == 0
for i in range (0, n_samples):
    
    partitions = partition_data[i]
    task = random.uniform(0, partitions.max())
    task_data[i] = task
    
    best_partition = -1
    best_fit = 999999999
    
    for j in range (0, n_partitions):
        current_fit = partitions[j] - task
        data[i,j] = abs(current_fit)
        if current_fit < 0:
            data[i, j + n_partitions] = 0
        else:
            data[i, j + n_partitions] = 1
        if current_fit > 0 and current_fit < best_fit:
            best_fit = current_fit
            best_partition = j
    
    labels[i][best_partition] = 1


In [3]:
# split data between train and test set 
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.20, random_state=42)

In [4]:
print(data)
print(labels)

[[5.56792269e-01 3.98267564e-01 7.20768704e-01 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [1.17988899e-02 4.29982863e-01 1.56589065e-02 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [8.40378282e-03 5.85688878e-02 7.63773756e-01 ... 1.00000000e+00
  1.00000000e+00 1.00000000e+00]
 ...
 [2.33384305e-01 6.26002034e-02 8.45825351e-02 ... 1.00000000e+00
  0.00000000e+00 1.00000000e+00]
 [3.10310920e-01 3.77374757e-01 3.02609223e-01 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [3.19203256e-04 5.15233275e-01 1.78808234e-01 ... 0.00000000e+00
  1.00000000e+00 1.00000000e+00]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [5]:
import tensorflow as tf

# Construct neural network
n_nodes = 64

model = Sequential()
model.add(Dense(n_nodes, input_dim=2*n_partitions, activation='relu', name='layerA'))
model.add(Dense(n_nodes, activation='relu', name='layerB'))
model.add(Dense(n_partitions, activation='softmax', name='softmax'))

# Configure a model for categorical classification. from https://www.tensorflow.org/guide/keras#train_and_evaluate
model.compile(optimizer=tf.train.RMSPropOptimizer(0.003),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=[tf.keras.metrics.categorical_accuracy])

In [6]:
batchsize = 256

model.fit(X_train, y_train, epochs=20, batch_size=batchsize)
score = model.evaluate(X_test, y_test, batch_size=batchsize)
print(score)

Epoch 1/20
800000/800000 [==============================] - 18s 22us/step - loss: 0.4668 - categorical_accuracy: 0.8168
Epoch 2/20
800000/800000 [==============================] - 18s 23us/step - loss: 0.2304 - categorical_accuracy: 0.9038
Epoch 3/20
800000/800000 [==============================] - 18s 23us/step - loss: 0.1774 - categorical_accuracy: 0.9253
Epoch 4/20
800000/800000 [==============================] - 18s 23us/step - loss: 0.1445 - categorical_accuracy: 0.9393
Epoch 5/20
800000/800000 [==============================] - 18s 23us/step - loss: 0.1256 - categorical_accuracy: 0.9473
Epoch 6/20
800000/800000 [==============================] - 18s 22us/step - loss: 0.1128 - categorical_accuracy: 0.9529
Epoch 7/20
800000/800000 [==============================] - 17s 21us/step - loss: 0.1041 - categorical_accuracy: 0.9568
Epoch 8/20
800000/800000 [==============================] - 18s 22us/step - loss: 0.0972 - categorical_accuracy: 0.9601
Epoch 9/20
800000/800000 [==============

In [7]:
from keras.models import load_model

model.save('simple_classifer_sign_inputs.h5')  # creates a HDF5 file 'my_model.h5'
model.save_weights('simple_classifer_sign_inputs_wts.h5')

C:\Users\ortci\Anaconda3\lib\site-packages\keras\engine\saving.py:118: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


In [8]:
label_pred = model.predict(X_test)

In [14]:
# find cases for misclassification
misclass_set = []
for i in range(0,100):
    if (np.around(label_pred[i],0) != labels[i]).any():
        misclass_set.append(i)
num_misclass = len(misclass_set)
print(num_misclass)

5


In [15]:
# show results of any specific case
test = misclass_set[0]

import pandas as pd

test_df = pd.DataFrame(np.vstack((label_pred[test],X_test[test][0:n_partitions],X_test[test][n_partitions:2*n_partitions],y_test[test])))
test_df.index = ("Model Choice Prob", "Abs(fit)", "Fit?", "Label")
test_df.style.format("{:.2%}")

test_df

,0,1,2,3,4,5,6,7,8,9
Model Choice Prob,1.316627e-19,1.743895e-19,1.121482e-17,1.180437e-20,0.133849,5.798911e-20,0.447551,0.418600,2.055777e-17,2.224409e-19
Abs(fit),4.275934e-01,6.346387e-01,2.754248e-01,2.535166e-01,0.524839,2.112698e-01,0.625913,0.675556,5.801918e-01,3.399895e-01
Fit?,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00
Label,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00
